In [1]:
from qiskit import QuantumCircuit
import qiskit.circuit as qc
from qiskit import QuantumRegister
from qiskit_ibm_provider import IBMProvider
from qiskit import transpile
import numpy as np
import math

provider = IBMProvider()
backend = provider.get_backend('ibm_perth')


In [31]:
backend.properties().gate_error('cx',[0,1])

0.018198344236854097

In [57]:
import random
from qiskit import QuantumRegister
from qiskit import QuantumCircuit
from qiskit import Aer
from qiskit import transpile
def ising_1d_instance(qubits):
    def rand1d(qubits):
        return [random.choice([+1, -1]) for _ in range(qubits)]

    # transverse field terms
    h = rand1d(qubits)
    # links between lines
    j = rand1d(qubits-1)
    return (h, j)

nr_of_qubits = 4
h,j = ising_1d_instance(nr_of_qubits)
qr = QuantumRegister(nr_of_qubits)
circuit = QuantumCircuit(qr)
circuit.h([0])
circuit.h([1])
circuit.h([2])
circuit.h([3])
circuit.x(qubit=[0])
print(circuit)

def add_measurement(circuit, qubits):
    # Create a Quantum Circuit
    meas = QuantumCircuit(qubits, qubits)
    meas.barrier(range(qubits))
    # map the quantum measurement to the classical bits
    meas.measure(range(qubits), range(qubits))

    # The Qiskit circuit object supports composition using
    # the compose method.
    circuit.add_register(meas.cregs[0])
    qc = circuit.compose(meas)
    return qc

qc = add_measurement(circuit, nr_of_qubits)
print(qc)

backend_sim = Aer.get_backend('qasm_simulator')
counts = backend_sim.run(transpile(qc, backend_sim), shots=1024).result().get_counts()

print(counts)

       ┌───┐┌───┐
q44_0: ┤ H ├┤ X ├
       ├───┤└───┘
q44_1: ┤ H ├─────
       ├───┤     
q44_2: ┤ H ├─────
       ├───┤     
q44_3: ┤ H ├─────
       └───┘     
       ┌───┐┌───┐ ░ ┌─┐         
q44_0: ┤ H ├┤ X ├─░─┤M├─────────
       ├───┤└───┘ ░ └╥┘┌─┐      
q44_1: ┤ H ├──────░──╫─┤M├──────
       ├───┤      ░  ║ └╥┘┌─┐   
q44_2: ┤ H ├──────░──╫──╫─┤M├───
       ├───┤      ░  ║  ║ └╥┘┌─┐
q44_3: ┤ H ├──────░──╫──╫──╫─┤M├
       └───┘      ░  ║  ║  ║ └╥┘
  c: 4/══════════════╩══╩══╩══╩═
                     0  1  2  3 
{'1010': 60, '1111': 68, '0001': 56, '0000': 65, '0010': 60, '1110': 64, '1011': 57, '0111': 69, '0011': 64, '1100': 57, '0101': 77, '1001': 57, '1000': 64, '0110': 65, '1101': 66, '0100': 75}


In [62]:
shots = 1024

def compute_expected_energy(counts,h,j,shots):
    def bool_to_state(integer):
        # Convert the 1/0 of a bit to +1/-1
        return 2*int(integer)-1

    # Get total energy of each count
    r1=list(counts.keys())
    r2=list(counts.values())
    total_energy = 0

    for k in range(0,len(r1)):
        # r2[k] is the number of shots that have this result
        # r1[k] is the result as qubits (like 0001)
        
        # Energy of h
        for bit_h in range(0,len(r1[k])):
            total_energy += bool_to_state(r1[k][bit_h]) * h[bit_h]*r2[k]
        # Energy of j
        for bit_value in range(0,len(r1[k])-1):
            total_energy += bool_to_state(r1[k][bit_value])*bool_to_state(r1[k][bit_value+1])*j[bit_value]*r2[k]
    expectation_value = total_energy/shots
    return expectation_value

# 0001 -> -1,-1,-1,1
# ij_1 -1*-1*j[0]



# def digit_sum(n):
#     num_str = str(n)
#     sum = 0
#     for i in range(0, len(num_str)):
#         sum += int(num_str[i])
#     return sum


# print(r1)
# print(r2)
# for k in range(0,len(r1)):
#         expectation_value+=(4-2*digit_sum(r1[j]))*r2[j]/shots
# print(expectation_value)

1024


In [83]:
from qiskit.circuit import QuantumCircuit, Parameter
import numpy as np
phi = Parameter("1")
phi2 = Parameter("2")
qr = QuantumRegister(nr_of_qubits)
circuit = QuantumCircuit(qr)
circuit.h([0])
circuit.h([1])
circuit.h([2])
circuit.h([3])
circuit.rx(phi,qubit=[0])
circuit.rx(phi2,qubit=[1])
print(circuit)
print(circuit.parameters)
value = np.random.random()
parameters = [value for _ in range(2)]
print(circuit.bind_parameters(parameters))

       ┌───┐┌───────┐
q68_0: ┤ H ├┤ Rx(1) ├
       ├───┤├───────┤
q68_1: ┤ H ├┤ Rx(2) ├
       ├───┤└───────┘
q68_2: ┤ H ├─────────
       ├───┤         
q68_3: ┤ H ├─────────
       └───┘         
ParameterView([Parameter(1), Parameter(2)])
       ┌───┐┌───────────────────────┐
q68_0: ┤ H ├┤ Rx(0.349398938987163) ├
       ├───┤├───────────────────────┤
q68_1: ┤ H ├┤ Rx(0.349398938987163) ├
       ├───┤└───────────────────────┘
q68_2: ┤ H ├─────────────────────────
       ├───┤                         
q68_3: ┤ H ├─────────────────────────
       └───┘                         


In [84]:
from CLQGA import genome_to_circuit, compute_gradient

In [105]:
# TEST for gradient computation
CIRCUIT_DEPTH = 6
GATE_ENCODING_LENGTH = 6 
QUBIT_SECTIONING_LENGTH = 5
NR_OF_QUBITS = 5
NR_OF_GATES = CIRCUIT_DEPTH
NR_OF_ISING_QUBITS = 4
CHROMOSOME_LENGTH = CIRCUIT_DEPTH * (GATE_ENCODING_LENGTH + QUBIT_SECTIONING_LENGTH)
genome = ''.join(str(random.randint(0,1)) for _ in range(CHROMOSOME_LENGTH))
print(genome)
circuit, nr_of_parameters = genome_to_circuit(genome, NR_OF_QUBITS, NR_OF_GATES)
print(circuit)
observable_h, observable_j = ising_1d_instance(NR_OF_ISING_QUBITS)
gradient, circuit = compute_gradient(circuit, nr_of_parameters, NR_OF_ISING_QUBITS, observable_h, observable_j, 1024)
print(gradient)
print(circuit)

101111010110000111011001100100010111011000110100100001011101101110
                    
q206_0: ────────────
        ┌──────────┐
q206_1: ┤ U(0,1,2) ├
        └──────────┘
q206_2: ────────────
                    
q206_3: ────────────
                    
q206_4: ────────────
                    
{'0000': 117, '0010': 907}
total_energy =  -4418
EXPECTED ENERGY =  -4.314453125
{'0010': 932, '0000': 92}
total_energy =  -4568
EXPECTED ENERGY =  -4.4609375
{'0000': 104, '0010': 920}
total_energy =  -4496
EXPECTED ENERGY =  -4.390625
{'0000': 99, '0010': 925}
total_energy =  -4526
EXPECTED ENERGY =  -4.419921875
{'0000': 108, '0010': 916}
total_energy =  -4472
EXPECTED ENERGY =  -4.3671875
{'0000': 94, '0010': 930}
total_energy =  -4556
EXPECTED ENERGY =  -4.44921875
257.8125
                                                                 
q206_0: ─────────────────────────────────────────────────────────
        ┌───────────────────────────────────────────────────────┐
q206_1: ┤ U(3.786418